In [1]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd

In [2]:
dataset=pd.read_csv("CKD.csv")

In [3]:
dataset = pd.get_dummies(dataset, dtype=int, drop_first=True)

In [4]:
dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [5]:
dataset["classification_yes"].value_counts()

classification_yes
1    249
0    150
Name: count, dtype: int64

In [6]:
indep=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
dep=dataset[['classification_yes']]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(indep, dep, test_size=1/3, random_state=0)

In [10]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

param_grid={'priors':[None],
           'var_smoothing':[1e-09]}

grid=GridSearchCV(GaussianNB(),param_grid, refit=True, verbose=3, n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=GaussianNB(), n_jobs=-1,
             param_grid={'priors': [None], 'var_smoothing': [1e-09]},
             scoring='f1_weighted', verbose=3)

In [11]:
print(grid.best_params_)

{'priors': None, 'var_smoothing': 1e-09}


In [12]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test, grid_predictions)

In [14]:
from sklearn.metrics import classification_report
clf_report=classification_report(y_test, grid_predictions)

In [15]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions, average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'priors': None, 'var_smoothing': 1e-09}: 0.9775556904684072


In [16]:
print("The Confusion Matrix:\n",cm)

The Confusion Matrix:
 [[51  0]
 [ 3 79]]


In [17]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97        51
           1       1.00      0.96      0.98        82

    accuracy                           0.98       133
   macro avg       0.97      0.98      0.98       133
weighted avg       0.98      0.98      0.98       133



In [18]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

1.0

In [19]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_priors,param_var_smoothing,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.014074,0.007252,0.009157,0.001685,None,1.000000e-09,"{'priors': None, 'var_smoothing': 1e-09}",0.981569,0.981014,0.981217,1.0,0.981217,0.985003,0.0075,1


In [20]:
Future_Prediction=grid.predict([[10,70,2,0,200,50,2,130,4,8,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1]]) 
print("Future_Prediction={}".format(Future_Prediction))

Future_Prediction=[1]


C:\Anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
